In [ ]:
import re
from tqdm import tqdm

## Download DB

In [ ]:
!gdown 1IAVWMAYGz-f3jO_WvTsRyWwgmbdIxlNm

Downloading...
From: https://drive.google.com/uc?id=1IAVWMAYGz-f3jO_WvTsRyWwgmbdIxlNm
To: /content/knowledge_base_kyivstar.md
100% 307k/307k [00:00<00:00, 75.5MB/s]


In [ ]:
with open("knowledge_base_kyivstar.md", 'r') as f:
  content = f.read().replace("\*\*", "**")

with open("knowledge_base_kyivstar.md", 'w') as f:
  f.write(content)

write files with questions that were parsed by Victor and Kostya (they have different format)

In [ ]:
!tail +220 knowledge_base_kyivstar.md > victors.md

In [ ]:
!head -n 220 knowledge_base_kyivstar.md > kostyas.md

parse Victors questions to list

In [ ]:
with open("victors.md", "r") as f:
  content = f.read()

In [ ]:
starts = [m.start() for m in re.finditer(r"^\d+\. \*\*", content, re.M)]

In [ ]:
questions_v = []

for i, start in enumerate(starts):
  next_start = starts[i + 1] if i + 1 != len(starts) else -1
  question = content[start:next_start].strip()

  question = re.sub(r"\[https.+?\]\(https.+?\)", "", question)
  question = "\n".join([i for i in question.split("\n") if i.strip()])
  questions_v.append(question)

In [ ]:
len(questions_v)

250

parse Kostyas questions to list

In [ ]:
with open("kostyas.md", "r") as f:
  content = f.read()

In [ ]:
starts = [m.start() for m in re.finditer(r"^\t?(  )?\d+\\?\. \*\*", content, re.M)]

In [ ]:
questions_k = []

for i, start in enumerate(starts):
  next_start = starts[i + 1] if i + 1 != len(starts) else -1
  question = content[start:next_start].strip()
  questions_k.append(question)

In [ ]:
len(questions_k)

49

Combine

In [ ]:
questions = []
questions += questions_k
questions += questions_v

In [ ]:
len(questions)

299

In [ ]:
questions[:10]

['1\\. **Які є тарифи тільки для дзвінків?**   \nПропонуємо подивитися умови тарифу LOVE UA На максимум з підключеною Суперсилою Економія. У вас буде безлім на Київстар, 500 МБ Інтернету та ще одна Суперсилу на вибір. Вартість тарифу — 125 грн/4 тижні (заощаджуйте 50 грн, від звичайної вартості 175 грн/4 тижні).',
 '2\\. **Як перевірити мобільний рахунок?** Надішліть запит \\*111\\#. Так само можна перевірити рахунок у роумінгу. Скористайтеся додатком Мій Київстар.',
 "3\\. **Що робити, якщо кошти випадково пішли не на той номер?** Якомога швидше напишіть нам у чат. Будь ласка, зауважте, що абонентам передплаченого зв'язку наразі технічно неможливо повернути помилковий платіж. Існує низка технічних обмежень, що не дозволяють компанії повернути помилковий платіж. Наприклад, якщо одержувач уже використав зараховані кошти. Відкликати платіж може касир або служба підтримки термінала і лише в день оплати.",
 '4\\. **Як перенести гроші на інший мобільний номер?** З цією технологічною можливі

In [ ]:
import pickle

with open('questions.pickle', 'wb') as handle:
    pickle.dump(questions, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('questions.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(questions == b)

True


## END OF generating questions list

## Get embeddings

In [ ]:
!pip install -q httpx==0.27.2
!pip install -q openai
!pip install -q tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.3 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import tiktoken

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def price_of_embedding_request(string: str) -> float:
  # https://openai.com/api/pricing/#:~:text=text%2Dembedding%2D3,0.020%20/%201M%20tokens
  price_per_token = 0.02 / 1_000_000
  tokens = num_tokens_from_string(string)
  return tokens * price_per_token

In [ ]:
total_price = 0

for question in tqdm(questions):
  total_price += price_of_embedding_request(question)

print()
print(f"${total_price}" )

100%|██████████| 299/299 [00:00<00:00, 5153.39it/s]


$0.0016315200000000002


In [ ]:
embeddings = []

for question in tqdm(questions):
  embedding = get_embedding(question)
  embeddings.append(embedding)

In [ ]:
import pickle

with open('embeddings.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('embeddings.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(embeddings == b)

True
